# Classification Dataset

### Data Mining Project 2024/25

Authors: Nicola Emmolo, Simone Marzeddu, Jacopo Raffi

In [13]:
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.over_sampling import ADASYN
from scipy.stats import ks_2samp

import pandas as pd
import numpy 
from sklearn.model_selection import train_test_split
import seaborn as sb

In [14]:
dataset = pd.read_csv('../../data/complete_dataset_no_outliers.csv', dtype={'cyclist_team': str})
RANDOM_STATE = 42   

In [15]:
dataset['label'] = (dataset['raw_position'] <= 20).astype(int) #1 if top 20, 0 otherwise 

In [16]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565555 entries, 0 to 565554
Data columns (total 38 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   cyclist_url                  565555 non-null  object 
 1   cyclist_name                 565555 non-null  object 
 2   birth_year                   565555 non-null  int64  
 3   weight                       455823 non-null  float64
 4   height                       455823 non-null  float64
 5   nationality                  565555 non-null  object 
 6   race_url                     565555 non-null  object 
 7   race_name                    565555 non-null  object 
 8   points                       565555 non-null  float64
 9   uci_points                   398000 non-null  float64
 10  length                       565555 non-null  float64
 11  climb_total                  451154 non-null  float64
 12  profile                      451154 non-null  float64
 13 

Feature da eliminare:

- cyclist_url (non generalizzabili)
- cyclist_name (non generalizzabili)
- birth_year (gia incluso in age)
- nationality (troppo sbilanciamento nel dataset)
- weight, height (gia incluso in BMI)
- uci_points (troppo pochi e gia inclusi in points)
- race_url (non generalizzabili)
- race_name (non generalizzabili)
- climb_total, profile (gia inclusi in race_physical_effort)
- mostly_tarmac (sbilanciato)
- cyclist_team (non generalizzabili)
- date (non generalizzabili)
- raw_position, position (obv)
- delta (obv) 
- race_year (non generalizzabili)
- race_stage (non generalizzabili)
- cyclist_age_group (??)
- cyclist_climb_power (feature generata con delta, obv)
- points, startlist_quality (gia inclusti in race_prestige)

We must choose between cyclist_age and cyclist_age_group as both these features represent the same information bu give different advantage/disadvantage to our classification process: 
- cyclist_age -> lower dimensionality (no one-hot) but unbalanced representation
- cyclist_age_group -> arbitraty encoding but balanced representation 

we chose to delete cyclist_age as the encoding is arbitraty but it still respects the "order" of the age_groups

In [17]:
columns = ['length', 'race_season', 'cyclist_bmi', 'cyclist_age_group', 'climb_percentage', 'race_physical_effort', 'race_prestige', 'race_year', 
           'previous_mean_position', 'previous_mean_delta', 'previous_mean_cp', 'cyclist_previous_experience', 'label']
dataset = dataset[columns].dropna()

In [18]:
development_dataset = dataset[dataset['race_year'] < 2022]
test_dataset = dataset[dataset['race_year'] >= 2022]

development_dataset = development_dataset.drop(columns=['race_year'])
test_set = test_dataset.drop(columns=['race_year'])
test_label = test_set.pop('label')

In [19]:
label = development_dataset.pop('label')
train_set, val_set, train_label, val_label = train_test_split(development_dataset, label, stratify=label, test_size=0.20, random_state=RANDOM_STATE)

In [20]:
development_set = development_dataset.copy()
developmente_label = label

In [21]:
developmente_label.value_counts(normalize=True)

label
0    0.863163
1    0.136837
Name: proportion, dtype: float64

In [22]:
development_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 332469 entries, 5 to 565553
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   length                       332469 non-null  float64
 1   race_season                  332469 non-null  object 
 2   cyclist_bmi                  332469 non-null  float64
 3   cyclist_age_group            332469 non-null  object 
 4   climb_percentage             332469 non-null  float64
 5   race_physical_effort         332469 non-null  float64
 6   race_prestige                332469 non-null  float64
 7   previous_mean_position       332469 non-null  float64
 8   previous_mean_delta          332469 non-null  float64
 9   previous_mean_cp             332469 non-null  float64
 10  cyclist_previous_experience  332469 non-null  float64
dtypes: float64(9), object(2)
memory usage: 30.4+ MB


In [23]:
def discretize_data(dataset, variables):
    df = dataset.copy()
    for variable in variables:
        #get the unique variable's values
        var = sorted(df[variable].unique())
        
        #generate a mapping from the variable's values to the number representation  
        mapping = dict(zip(var, range(0, len(var) + 1)))

        #add a new colum with the number representation of the variable
        df[variable] = df[variable].map(mapping).astype(int)
    return df

In [24]:
categorical_features = dataset.select_dtypes(exclude="number").columns.tolist()
models = [
    RandomOverSampler(random_state=RANDOM_STATE),
    SMOTE(random_state=RANDOM_STATE, k_neighbors=10),
    SMOTENC(random_state=RANDOM_STATE, k_neighbors=10, categorical_features=['race_season']),
    ADASYN(random_state=RANDOM_STATE, n_neighbors=10)
]
oversampling_algorithms = [
    "random",
    "smote_interpolation",
    "smote_interpolation_w_categorical",
    "adasyn"
]
oversampled_datasets = list()

for algorithm, model in zip(oversampling_algorithms, models):
    if algorithm in ("smote_interpolation", "adasyn"):
        oversampled_data, oversampled_labels = model.fit_resample(development_set.select_dtypes(include="number"), developmente_label)
        oversampled_labels = oversampled_labels.to_numpy().reshape(-1, 1)
        oversampled_dataset = pd.DataFrame(numpy.hstack((oversampled_data, oversampled_labels)), columns=list(development_set.select_dtypes(include="number").columns) + ['label'])

    elif algorithm == "smote_interpolation_w_categorical":
        df = discretize_data(development_set, ['cyclist_age_group'])
        oversampled_data, oversampled_labels = model.fit_resample(df, developmente_label)
        oversampled_labels = oversampled_labels.to_numpy().reshape(-1, 1)
        oversampled_dataset = pd.DataFrame(numpy.hstack((oversampled_data, oversampled_labels)), columns=(list(df.columns) + ['label']))
    
    else:
        oversampled_data, oversampled_labels = model.fit_resample(development_set, developmente_label)
        oversampled_labels = oversampled_labels.to_numpy().reshape(-1, 1)
        oversampled_dataset = pd.DataFrame(numpy.hstack((oversampled_data, oversampled_labels)), columns=(list(development_set.columns) + ['label']))
    
    oversampled_dataset["algorithm"] = algorithm

    oversampled_datasets.append(oversampled_dataset)

In [25]:
tests_per_algorithm = list()
development_dataset['label'] = label
for algorithm, oversampled_dataset in zip(oversampling_algorithms, oversampled_datasets):
    print(algorithm)
    columns = oversampled_dataset.columns
    test_results = list()
    for column in columns:
        if column != "algorithm":
            if not (column == 'cyclist_age_group' and algorithm == "smote_interpolation_w_categorical"):
                test_results.append(ks_2samp(development_dataset[column], oversampled_dataset[column], alternative="two-sided"))
            else:
                df = discretize_data(development_dataset, ['cyclist_age_group'])
                test_results.append(ks_2samp(df[column], oversampled_dataset[column], alternative="two-sided"))

    test_data = [(
        test.statistic,
        test.pvalue,
        test.statistic_location
        )
        for test in test_results        
    ]
    test_data = pd.DataFrame(test_data, columns=["KS_test", "p_value", "margin"])
    test_data["algorithm"] = algorithm

    tests_per_algorithm.append(test_data)

validation = pd.concat(tests_per_algorithm, axis="rows")
validation.groupby("algorithm").describe()

random
smote_interpolation
smote_interpolation_w_categorical
adasyn


KS_test                                \
                                    count      mean       std       min   
algorithm                                                                 
adasyn                               10.0  0.077715  0.104548  0.007097   
random                               12.0  0.057083  0.102074  0.001167   
smote_interpolation                  10.0  0.080432  0.104495  0.007260   
smote_interpolation_w_categorical    12.0  0.075127  0.095587  0.007371   

                                                                           \
                                        25%       50%       75%       max   
algorithm                                                                   
adasyn                             0.022302  0.050010  0.075660  0.363671   
random                             0.005032  0.016066  0.065323  0.363163   
smote_interpolation                0.023902  0.050131  0.082926  0.363163   
smote_interpolation_w_categorical  0.028637  0.049924  0.077655  0.363163   

                                  p_value                                   \
                                    count          mean           std  min   
algorithm                                                                    
adasyn                               10.0  1.217101e-10  3.848810e-10  0.0   
random                               12.0  1.103441e-01  2.794585e-01  0.0   
smote_interpolation                  10.0  4.589355e-11  1.451282e-10  0.0   
smote_interpolation_w_categorical    12.0  1.924017e-11  6.664990e-11  0.0   

                                                                     \
                                   25%            50%           75%   
algorithm                                                             
adasyn                             0.0  7.146024e-272  6.590708e-71   
random                             0.0   5.884048e-27  7.605526e-03   
smote_interpolation                0.0  1.507149e-260  2.797975e-86   
smote_interpolation_w_categorical  0.0  2.313725e-257  1.249553e-88   

                                                 
                                            max  
algorithm                                        
adasyn                             1.217101e-09  
random                             9.361713e-01  
smote_interpolation                4.589355e-10  
smote_interpolation_w_categorical  2.308820e-10

Random oversampling seems to mantain the original distribution (p-value = 0.11)

In [26]:
dev_oversample = oversampled_datasets[0]
dev_oversample['label'].value_counts(normalize=True)

label
0    0.5
1    0.5
Name: proportion, dtype: float64

In [27]:
dev_label = dev_oversample.pop('label')

In [28]:
#drop the categorical variables since we don't need them anymore 
cat_columns = ['race_season']
dev_oversample_encoded = discretize_data(dev_oversample, ['cyclist_age_group'])
dev_oversample_encoded = pd.get_dummies(dev_oversample_encoded, columns = cat_columns, prefix_sep='%').drop(columns=['algorithm'])
dev_oversample_encoded

,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
0,135.0,22.256908,1,0.003244,0.006662,0.193636,0.82306,161.333333,0.000118,0.024713,False,True,False,False
1,100.0,22.256908,1,0.00398,0.004484,0.193636,0.780877,138.285714,0.001042,0.049022,False,True,False,False
2,199.0,22.256908,1,0.009281,0.041416,0.193636,0.738174,121.0,0.003588,0.071902,False,False,True,False
3,244.0,22.256908,1,0.006988,0.140635,0.193636,0.739753,128.111111,0.002734,0.09766,False,False,True,False
4,244.0,22.256908,1,0.00952,0.063869,0.193636,0.743873,115.3,0.003841,0.131921,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573945,220.0,20.069204,2,0.010786,0.117654,0.10743,0.475134,199.917197,0.017458,0.230765,False,True,False,False
573946,27.0,21.100705,1,0.010593,0.001739,0.078075,0.363851,228.868217,0.014588,0.393118,False,False,False,True
573947,178.9,20.897959,4,0.016339,0.235699,0.065671,0.385872,345.306604,0.0071,0.483079,False,False,False,True
573948,184.8,20.60378,4,0.010162,0.117321,0.139441,0.555717,510.757576,0.000973,0.099031,False,False,True,False


In [29]:
test_set = discretize_data(test_set, ['cyclist_age_group'])
cat_columns = ['race_season']
test_set_encoded = pd.get_dummies(test_set, columns = cat_columns, prefix_sep='%')
test_set_encoded

,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
195,203.9,21.705129,3,0.009858,0.092363,0.225487,0.729362,470.333333,0.000047,0.009954,False,False,False,True
197,204.2,20.756387,0,0.008159,0.076668,0.264741,0.906142,591.500000,0.000036,0.002329,False,False,False,True
198,203.9,20.756387,0,0.009858,0.092363,0.225487,0.876822,643.666667,0.000031,0.003379,False,False,False,True
199,183.7,20.756387,0,0.008977,0.068267,0.228368,0.828177,559.000000,0.000032,0.006341,False,False,False,True
437,123.4,18.827160,3,0.018476,0.158518,0.000000,0.454167,211.500000,0.000211,0.000813,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565194,193.0,20.305175,4,0.018705,0.235529,0.078556,0.419099,469.501340,0.002286,0.600163,False,False,False,True
565195,154.0,20.305175,4,0.010779,0.028805,0.078556,0.420047,468.336898,0.002282,0.600797,False,False,False,True
565196,204.1,20.305175,4,0.009882,0.092776,0.304715,0.420876,467.380319,0.002274,0.746805,False,False,False,True
565197,273.4,20.305175,4,0.007992,0.134628,0.399068,0.421826,468.395225,0.002271,0.604920,False,True,False,False


Abbiamo trasformato gli attributi categorici in equivalenti numerici per poter essere processati dai modelli di learning: 
- one-hot encoding: è stato utilizzato quando l'attributo categorico non è di tipo ordinale, in questo caso non siamo interessati a preservare ordine o relazioni di vicinanza tra i valori di questo attributo e pertanto vogliamo massimizzare la dissimilarità tra le classi
- discretizzazione: abbiamo usato questa tecnica solo nel caso dell'attributo cyclist_age_group, in quanto queste classi possiedono relazioni di ordine e vicinanza (attributo ordinale)

In [30]:
dev_oversample_encoded['label'] = dev_label
test_set_encoded['label'] = test_label

In [31]:
dev_oversample_encoded.to_csv('../../data/ml_datasets/oversampling/dev_set.csv', index=False)
test_set_encoded.to_csv('../../data/ml_datasets/oversampling/test_set.csv', index=False)

: 